# Data Product Arquitecture - Food Inspections Project

Team:

* Jóse Roberto Pérez - 173013
* Santiago Battezzati - 197727
* Carlos Geovany Pérez - 197614
* Mariana Lugo - 118626

## Imports

#### Standard library imports

In [1]:
import os
import sys
from datetime import datetime
import re

#### Third party imports

In [2]:
import pandas as pd
from pandas_profiling import ProfileReport
pd.options.display.max_columns = 25

import matplotlib.pyplot as plt
import seaborn as sns

#### Local application imports

In [3]:
sys.path.append("../")

In [4]:
%reload_ext autoreload
%autoreload 2

from src.utils.utils import (
    json_dump_dict,
    load_df,
)

from src.etl.ingesta_almacenamiento import (
    initial_cleaning,
)

from src.utils.data_dict import data_dict

from src.utils.params_gen import (
    ingestion_pickle_loc,
    transformation_pickle_loc,
    cat_reduction_ref,
)

---

## Importing and cleaning data
Food Inspections in Chicago: https://data.cityofchicago.org/Health-Human-Services/Food-Inspections/4ijn-s7e5

In [5]:
df = pd.read_csv('../data/raw/Food_Inspections.csv')
df.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2453552,CLAMP DOWN BURGERS,NaN,2749943.0,NaN,Risk 2 (Medium),1742 W DIVISION ST,CHICAGO,IL,60622.0,10/20/2020,License,No Entry,NaN,41.903387,-87.671740,"(-87.67174026586648, 41.903386755553484)"
1,2386633,JIN JU,JIN JU,27137.0,Restaurant,Risk 1 (High),5203 N CLARK ST,CHICAGO,IL,60640.0,08/28/2020,Canvass,No Entry,NaN,41.976301,-87.668276,"(-87.66827593789948, 41.97630115368914)"
2,2386595,LA BIZNAGA #2,LA BIZNAGA #2,2708992.0,NaN,Risk 1 (High),2949 W BELMONT AVE,CHICAGO,IL,60618.0,08/27/2020,Complaint,No Entry,NaN,41.939256,-87.702270,"(-87.70226967930802, 41.939255926667535)"
3,2386464,Uni Sushi Bristo,Uni Sushi Bistro,2262637.0,Restaurant,Risk 1 (High),1752 W NORTH AVE,CHICAGO,IL,60622.0,08/25/2020,Complaint,No Entry,NaN,41.910676,-87.672205,"(-87.67220465807979, 41.91067561170382)"
4,2386398,KIKI'S BISTRO,KIKI'S BISTRO,22899.0,Restaurant,Risk 1 (High),900 N FRANKLIN ST,CHICAGO,IL,60610.0,08/24/2020,Canvass,No Entry,NaN,41.898998,-87.635921,"(-87.63592067312285, 41.89899799424835)"


**Limpieza de nombres de las columnas:**

In [ ]:
clean_col_names(df)

### Exploración de variables

**Número de variables:**

In [ ]:
count_vars(df)

**Número de observaciones:**

In [ ]:
count_obs(df)

**¿Qué variables son?**

In [ ]:
count_unique_obs(df)

In [ ]:
cat_vars=['dba_name','aka_name','facility_type','risk','address','city','state','inspection_type',
          'results','violations']
data_profiling_categ(df, cat_vars)

### Creación del diccionario de definiciones

In [ ]:
json_dump_dict({col: {"relevant": False, "data_type": "categoric", "model_relevant": False} for col in df.columns})

In [ ]:
{col: {"relevant": False}, {"data_type": "categoric"} for col in df.columns}

### Análisis de texto
Issue #13 GitHub

#### 1. Limpieza de campo `facility_type`

In [ ]:
print(len(dfx["facility_type"].unique()))
dfx["facility_type"].unique()

In [ ]:
dfxx = dfx["facility_type"].value_counts().to_frame()
dfxx[:50]

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 7))


## Distplot
ax[0].set(xlim=(0, 40e3))
sns.distplot(dfx["facility_type"], ax=ax[0], color="red")

## Boxplot
ax[1].set(ylim=(0, 75))
sns.boxplot(data=dfx["facility_type"], color="red", ax=ax[1])


plt.show()

In [ ]:
dfx["facility_type"].fillna("nan", inplace=True)

dfx["facility_type"] = dfx["facility_type"].apply(lambda x: clean_txt(x))

In [ ]:
print(len(dfx["facility_type"].unique()))
dfx["facility_type"].value_counts()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 7))


## Distplot
ax[0].set(xlim=(0, 40e3))
sns.distplot(dfx["facility_type"], ax=ax[0], color="blue")

## Boxplot
ax[1].set(ylim=(0, 25))
sns.boxplot(data=dfx["facility_type"], color="blue", ax=ax[1])


plt.show()

In [ ]:
# mr1 = dfx["facility_type"] <= 1
mr2 = dfx.index.str.contains("rest")
dfx.loc[(mr2), :][:50]

**Notas de análisis:**
- @9/Feb/2020
    - Una vez limpiados los textos, la cantidad de únicos pasó de 501 a 448.
    - Tal vez podríamos agrupar las categorías por aparición de palabras clave (e.g. `daycare` OR `day_care`).
    - Todas las que caigan fuera de las categorías, serán incluidas en una reserva llamada "Otros"
    - Podría valer la pena hacer un análisis de clústering para agrupar palabras por similitudes y así definir las categorías. (tal vez va a ser difícil hacer estas asociaciones.)

#### 2. Limpieza de campo `inspection_type`

In [ ]:
print(len(df["inspection_type"].unique()))
df["inspection_type"].unique()

In [ ]:
dfx = df["inspection_type"].value_counts().to_frame()
dfx

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 7))


## Distplot
ax[0].set(xlim=(0, 40e3))
sns.distplot(dfx["inspection_type"], ax=ax[0], color="red")

## Boxplot
ax[1].set(ylim=(0, 15))
sns.boxplot(data=dfx["inspection_type"], color="red", ax=ax[1])


plt.show()

In [ ]:
df["inspection_type"].fillna("nan", inplace=True)

df["inspection_type"] = df["inspection_type"].apply(lambda x: clean_txt(x))

In [ ]:
print(len(df["inspection_type"].unique()))
df["inspection_type"].value_counts()

In [ ]:
dfx = df["inspection_type"].value_counts().to_frame()
dfx

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 7))


## Distplot
ax[0].set(xlim=(0, 40e3))
sns.distplot(dfx["inspection_type"], ax=ax[0], color="blue")

## Boxplot
ax[1].set(ylim=(0, 25))
sns.boxplot(data=dfx["inspection_type"], color="blue", ax=ax[1])


plt.show()

In [ ]:
dfx

In [ ]:
# mr1 = dfx["facility_type"] <= 1
mr2 = dfx.index.str.contains("canvass")
dfx.loc[(mr2), :][:50]

In [ ]:
# mr1 = dfx["facility_type"] <= 1
mr2 = dfx.index.str.contains("license")
dfx.loc[(mr2), :][:50]

**Notas de análisis:**
- @7/Mar/2021
    - Una vez limpiados los textos, la cantidad de únicos pasó de 111 a 99.
    - Tal vez podríamos agrupar las categorías por aparición de palabras clave (e.g. `canvass` OR `license`).
    - Todas las que caigan fuera de las categorías, serán incluidas en una reserva llamada "Otros"
    - Podría valer la pena hacer un análisis de clústering para agrupar palabras por similitudes y así definir las categorías. (tal vez va a ser difícil hacer estas asociaciones.)

#### 3. Limpieza de campo `violations`

In [ ]:
dfx

In [ ]:
mr1 = dfx["violations"] != "nan"
dfx.loc[mr1, ["violations"]].values[1]

In [ ]:
dfx = df.copy()

dfx["violations"] = "| " + dfx["violations"]

dfx

In [ ]:
mr1 = dfx["violations"].notnull()
text_x = dfx.loc[mr1, ["violations"]].values[1][0]
text_x

In [ ]:
v_nums = re.findall('\| (.+?). ', text_x)
# v_nums = []
v_nums

In [ ]:
serious_vs = [str(num) for num in list(range(1, 44 + 1)) + [70]]
serious_vs

In [ ]:
set_x = set(serious_vs) - set(v_nums)
set_x

In [ ]:
def mark_serious_violations(row):
    
    try:
        
        v_nums = re.findall('\| (.+?). ', row)
        
        if len(set(serious_vs) - set(v_nums)) == len(set(serious_vs)):
            res = "no_serious_violations"
        
        else:
            res = "serious_violations"
            
    except:
        res = "no_result"
        
    return res

In [ ]:
dfx["serious_violations"] = dfx["violations"].apply(lambda x: mark_serious_violations(x))

---

## Exploration based on questions

### 1. ¿Cuáles son los tipos 10 de establecimientos que más violaciones cometen? (Facility Type)


In [ ]:
conteos_ft=df.groupby(["results","facility_type"]).size().reset_index(name="counts")
conteos_ft=conteos_ft.pivot("facility_type","results","counts")
conteos_ft=conteos_ft.fillna(0)
conteos_ft['Total']=conteos_ft.sum(1)
conteos_ft =conteos_ft.sort_values('Fail',ascending=False)
conteos_ft

In [ ]:
n = 10
top10=df['facility_type'].value_counts()[:10].sort_values(ascending=False)
top10

Se muestra el porcentaje del tipo de establecimiento acuerdo con el resultado de la inspección:

In [ ]:
proporciones_ft=round(conteos_ft.iloc[:,0:7].apply(lambda x: x/x.sum()*100, axis=1),2)
proporciones_ft=proporciones_ft.sort_values('Fail',ascending=False)
proporciones_ft

### 2. ¿Qué tipo de inspección detecta más establecimientos fuera de regla? (Inspection Type)

In [ ]:
conteos_it=df.groupby(["results","inspection_type"]).size().reset_index(name="counts")
conteos_it=conteos_it.pivot("inspection_type","results","counts")
conteos_it=conteos_it.fillna(0)
conteos_it['Total']=conteos_it.sum(1)
conteos_it =conteos_it.sort_values('Fail',ascending=False)
conteos_it

Se muestra el porcentaje del tipo de inspección de acuerdo con los resultados:

In [ ]:
proporciones_it=round(conteos_it.iloc[:,0:7].apply(lambda x: x/x.sum()*100, axis=1),2)
proporciones_it=proporciones_it.sort_values('Fail',ascending=False)
proporciones_it

Se muestra el top 20 de tipos de inspección con un porcentaje mayor de inspecciones no reprobadas.

In [ ]:
top20=proporciones_it.iloc[0:19,:]
top20

### 3. Relación entre el riesgo y resultado, ¿los más riesgosos no pasan la insepección?

In [ ]:
rr=df.groupby(['results','risk']).size().reset_index(name="counts")
rr=rr.pivot("results", "risk","counts")
rr=rr.fillna(0)
rr['Total']=rr.sum(1)
rr =rr.sort_values('Risk 1 (High)',ascending=False)
rr

In [ ]:
prop=round(rr.iloc[:,0:4].apply(lambda x: x/x.sum()*100, axis=1),2)
prop

In [ ]:
prop.plot(kind='bar',stacked=True)

### 4. ¿Cómo es el comportamiento de las inspecciones en el tiempo?

Se separa inspection_date en tres diferentes columnas: day, month, year y las agrega al dataframe:

In [ ]:
fecha= df.inspection_date.str.split("/", n=2,expand=True)
df['day']=fecha[1]
df['month']=fecha[0]
df['year']=fecha[2]

In [ ]:
df['inspection_date'] = pd.to_datetime(df['inspection_date'])

In [ ]:
# Columnas de year, month y counts
fecha_separated = df.groupby(['year','month']).size().reset_index(name="conteos")
# Hacemos tabla donde las columnas son años y las filas mes 
fecha_separated_p = fecha_separated.pivot("month", "year","conteos")
# Graficamos 
sns.set(style='whitegrid',rc={'figure.figsize':(10,10)},palette='Set3')
h =sns.lineplot(data=fecha_separated_p,linewidth = 3,dashes=False)
plt.legend(fontsize='x-large', title_fontsize='40',loc='lower right')
h.set(xlabel='Meses',ylabel='Conteos de Inspecciones')
plt.show()

### 5. ¿Cómo es el comportamiento por tipo de resultados en el tiempo?

In [ ]:
fecha_separated2 = df.groupby(['year','month','results']).size().reset_index(name="counts")
fecha_separated2['date']= fecha_separated2['year']+'-'+fecha_separated2['month']
g=fecha_separated2.drop(['year','month'],1)
g=g.set_index('date')
sns.set(style='whitegrid',rc={'figure.figsize':(20,10)},palette='Set2')
sns.lineplot(x='date',y='counts',hue='results',data=g,linewidth = 2)
plt.xticks(rotation= 90,size=8)
plt.show()

Se muestra el comportamiento anterior en porcentaje de inspecciones:

In [ ]:
g2=fecha_separated2.drop(['year','month'],1)
g2=g2.pivot("date", "results","counts")
g2=g2.fillna(0)
g2['total']=g2.sum(1)
prop2=round(g2.iloc[:,0:5].apply(lambda x: x/x.sum()*100, axis=1),2)
sns.set(style='whitegrid',rc={'figure.figsize':(20,10)},palette='Set2')
h =sns.lineplot(data=prop2,linewidth = 2,dashes=False,marker='o')
plt.xticks(rotation= 90)
plt.legend(fontsize='x-large', title_fontsize='40')
plt.show()

### 6. ¿Es importante la temporalidad de año, mes y día para las inspecciones?

Con variable inspection_date en formato tiempo creamos conteos del número de inspecciones hechas por día:

In [ ]:
conteos_fechas = df.groupby(['inspection_date']).size().reset_index(name='conteos')
# hacemos que inspection_date sea el identificador de las filas
conteos_fechas = conteos_fechas.set_index('inspection_date')


In [ ]:
# Separamos los días, meses, años
conteos_fechas.insert(1, 'year', conteos_fechas.index.year)
conteos_fechas.insert(2, 'month', conteos_fechas.index.month)
conteos_fechas.insert(3, 'day', conteos_fechas.index.day)
conteos_fechas = conteos_fechas.copy()

In [ ]:
# Graficamos
fig, axs = plt.subplots(figsize=(30,20),ncols=6, nrows=2)
heatmaps(conteos_fechas,2010,axs[0][0],'no','Day')
heatmaps(conteos_fechas,2011,axs[0][1],'no','')
heatmaps(conteos_fechas,2012,axs[0][2],'no','')
heatmaps(conteos_fechas,2013,axs[0][3],'no','')
heatmaps(conteos_fechas,2014,axs[0][4],'no','')
heatmaps(conteos_fechas,2015,axs[0][5],'yes','')
heatmaps(conteos_fechas,2016,axs[1][0],'no','')
heatmaps(conteos_fechas,2017,axs[1][1],'no','')
heatmaps(conteos_fechas,2018,axs[1][2],'no','')
heatmaps(conteos_fechas,2019,axs[1][3],'no','')
heatmaps(conteos_fechas,2020,axs[1][4],'no','')
heatmaps(conteos_fechas,2021,axs[1][5],'yes','')

**Vemos si para si existe un patrón temporal en el universo de inspecciones que resultaron como reprobadas (Results="Fail"):**

In [ ]:
fail=df[df.results=="Fail"]
len(fail)

In [ ]:
conteos_f = fail.groupby(['inspection_date']).size().reset_index(name='conteos')
conteos_f= conteos_f.set_index('inspection_date')
conteos_f.insert(1, 'year', conteos_f.index.year)
conteos_f.insert(2, 'month', conteos_f.index.month)
conteos_f.insert(3, 'day', conteos_f.index.day)
conteos_f = conteos_f.copy()

In [ ]:
fig, axs = plt.subplots(figsize=(30,20),ncols=6, nrows=2)
heatmaps(conteos_f,2010,axs[0][0],'no','Day')
heatmaps(conteos_f,2011,axs[0][1],'no','')
heatmaps(conteos_f,2012,axs[0][2],'no','')
heatmaps(conteos_f,2013,axs[0][3],'no','')
heatmaps(conteos_f,2014,axs[0][4],'no','')
heatmaps(conteos_f,2015,axs[0][5],'yes','')
heatmaps(conteos_f,2016,axs[1][0],'no','Day')
heatmaps(conteos_f,2017,axs[1][1],'no','')
heatmaps(conteos_f,2018,axs[1][2],'no','')
heatmaps(conteos_f,2019,axs[1][3],'no','')
heatmaps(conteos_f,2020,axs[1][4],'yes','')
heatmaps(conteos_f,2021,axs[1][5],'yes','')

---

## Mapa

In [ ]:
sns.set(style='whitegrid',rc={'figure.figsize':(20,10)},palette='Set1')
s = sns.lmplot(x='longitude',y='latitude',data=df,hue='results',col='results',fit_reg=False,height=6,
               scatter_kws={"s":1,"alpha":.5})

In [ ]:
cols = ['Results','Latitude','Longitude']
rest = pd.read_csv('../data/Food_Inspections.csv', usecols=cols)

Eliminamos los registros sin coordenadas completas:

In [ ]:
rest = rest.dropna(subset=['Latitude'])
rest = rest.dropna(subset=['Longitude'])

Filtramos únicamente aquellos registros que pasaron o no pasaron la prueba

In [ ]:
passed = rest[rest['Results']=='Pass']
failed = rest[rest['Results']=='Fail']

In [ ]:
len(passed), len(failed)

Imagen de la ciudad de chicago:

In [ ]:
chicago = plt.imread('../images/chicago.png')

In [ ]:
boundires = ((rest.Longitude.min(),   rest.Longitude.max(),      
         rest.Latitude.min(), rest.Latitude.max()))

In [ ]:
fig, ax = plt.subplots(1,2, sharex=True, sharey=True, figsize = (20,20))

ax[0].scatter(passed.Longitude, passed.Latitude, zorder=2, alpha=0.1, c='b', s=2)
ax[0].imshow(chicago, zorder=0, extent=boundires, aspect='equal')
ax[0].title.set_text('Results: Pass')

ax[1].scatter(failed.Longitude, failed.Latitude, zorder=2, alpha=0.1, c='r', s=2)
ax[1].imshow(chicago, zorder=0, extent=boundires, aspect='equal')
ax[1].title.set_text('Results: Fail')

plt.show()

---

## **Notas**

### Evaluación de funciones

In [ ]:
dfx = initial_cleaning(df)

In [ ]:
dfx

In [ ]:
dfx["results"].value_counts()

In [ ]:
dfx["label"].value_counts()

---

### Exploración de resultados de módulos

In [16]:
dfx = load_df("../" + transformation_pickle_loc)
dfx

,facility_type,risk,city,inspection_type,results,violations,label,serious_violations
0,facility_type_other,risk_2_-medium-,chicago,license,no_entry,-_nan,0,no_result
1,restaurant_bar,risk_1_-high-,chicago,canvass,no_entry,-_nan,0,no_result
2,facility_type_other,risk_1_-high-,chicago,complaint,no_entry,-_nan,0,no_result
3,restaurant_bar,risk_1_-high-,chicago,complaint,no_entry,-_nan,0,no_result
4,restaurant_bar,risk_1_-high-,chicago,canvass,no_entry,-_nan,0,no_result
...,...,...,...,...,...,...,...,...
215062,restaurant_bar,risk_2_-medium-,chicago,license,fail,-_nan,0,no_result
215063,restaurant_bar,risk_1_-high-,chicago,license,fail,-_12._hand_washing_facilities-_with_soap_and_s...,0,serious_violations
215064,restaurant_bar,risk_1_-high-,chicago,license,pass,-_nan,1,no_result
215065,facility_type_other,risk_3_-low-,chicago,license,pass,-_nan,1,no_result


In [18]:
dfx["inspection_type"].value_counts(normalize=True)[:50]

canvass                  0.637267
license                  0.179386
complaint                0.127686
inspection               0.041917
inspection_type_other    0.013745
Name: inspection_type, dtype: float64

---
---